In [1]:
!pip install langchain langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 338.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 336.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.8/787.8 kB 234.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 187.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 267.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 313.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 164.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 278.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

In [3]:
OPENAI_API_KEY = "OPENAI_API_KEY"

In [4]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.1, api_key=OPENAI_API_KEY)

In [5]:
task_prompt = """
Your task is to create a Python function named `calculate_factorial`.
This function should do the following:
1.  Accept a single integer `n` as input.
2.  Calculate its factorial (n!).
3.  Include a clear docstring explaining what the function does.
4.  Handle edge cases: The factorial of 0 is 1.
5.  Handle invalid input: Raise a ValueError if the input is a negative number.
"""

In [6]:
# --- The Reflection Loop ---
max_iterations = 3
current_code = ""

# We will build a conversation history to provide context in each step.
message_history = [HumanMessage(content=task_prompt)]

for i in range(max_iterations):
    print("\n" + "="*25 + f" REFLECTION LOOP: ITERATION {i + 1} " + "="*25)
    # --- 1. GENERATE / REFINE STAGE ---
    # In the first iteration, it generates. In subsequent iterations, it refines.
    if i==0:
        print("\n>>> STAGE 1: GENERATING initial code...")
        # The first message is just the task prompt.
        response = llm.invoke(message_history)
        current_code = response.content
    else:
        print("\n>>> STAGE 1: REFINING code based on previous critique...")
        # The message history now contains the task, the last code, and the last critique.
        # We instruct the model to apply the critiques.
        message_history.append(HumanMessage(content="Please refine the code using the critiques provided."))
        response = llm.invoke(message_history)
        current_code = response.content

    print("\n--- Generated Code (v" + str(i + 1) + ") ---\n" + current_code)
    message_history.append(response) # Add the generated code to history

    # --- 2. REFLECT STAGE ---
    print("\n>>> STAGE 2: REFLECTING on the generated code...")

    # Create a specific prompt for the reflector agent.
    # This asks the model to act as a senior code reviewer.
    reflector_prompt = [
        SystemMessage(content="""
            You are a senior software engineer and an expert in Python.
            Your role is to perform a meticulous code review.
            Critically evaluate the provided Python code based on the original task requirements.
            Look for bugs, style issues, missing edge cases, and areas for improvement.
            If the code is perfect and meets all requirements, respond with the single phrase 'CODE_IS_PERFECT'.
            Otherwise, provide a bulleted list of your critiques.
        """),
        HumanMessage(content=f"Original Task:\n{task_prompt}\n\nCode to Review:\n{current_code}")
    ]
    critique_response = llm.invoke(reflector_prompt)
    critique = critique_response.content

    # --- 3. STOPPING CONDITION ---
    if "CODE_IS_PERFECT" in critique:
        print("\n--- Critique ---\nNo further critiques found. The code is satisfactory.")
        break

    print("\n--- Critique ---\n" + critique)
    # Add the critique to the history for the next refinement loop.
    message_history.append(HumanMessage(content=f"Critique of the previous code:\n{critique}"))


========================= REFLECTION LOOP: ITERATION 1 =========================

>>> STAGE 1: GENERATING initial code...

--- Generated Code (v1) ---
Here's a Python function named `calculate_factorial` that meets the specified requirements:

```python
def calculate_factorial(n):
    """
    Calculate the factorial of a non-negative integer n.

    The factorial of a non-negative integer n is the product of all positive integers less than or equal to n.
    The factorial of 0 is defined to be 1.

    Parameters:
    n (int): A non-negative integer whose factorial is to be calculated.

    Returns:
    int: The factorial of the input integer n.

    Raises:
    ValueError: If the input is a negative integer.
    """
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    
    if n == 0:
        return 1
    
    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
    
    return factorial
```

### Explanation:
1. **Docstring**: The f

In [7]:
# --- The Reflection Loop ---
max_iterations = 10
current_code = ""

# We will build a conversation history to provide context in each step.
message_history = [HumanMessage(content=task_prompt)]

for i in range(max_iterations):
    print("\n" + "="*25 + f" REFLECTION LOOP: ITERATION {i + 1} " + "="*25)
    # --- 1. GENERATE / REFINE STAGE ---
    # In the first iteration, it generates. In subsequent iterations, it refines.
    if i==0:
        print("\n>>> STAGE 1: GENERATING initial code...")
        # The first message is just the task prompt.
        response = llm.invoke(message_history)
        current_code = response.content
    else:
        print("\n>>> STAGE 1: REFINING code based on previous critique...")
        # The message history now contains the task, the last code, and the last critique.
        # We instruct the model to apply the critiques.
        message_history.append(HumanMessage(content="Please refine the code using the critiques provided."))
        response = llm.invoke(message_history)
        current_code = response.content

    print("\n--- Generated Code (v" + str(i + 1) + ") ---\n" + current_code)
    message_history.append(response) # Add the generated code to history

    # --- 2. REFLECT STAGE ---
    print("\n>>> STAGE 2: REFLECTING on the generated code...")

    # Create a specific prompt for the reflector agent.
    # This asks the model to act as a senior code reviewer.
    reflector_prompt = [
        SystemMessage(content="""
            You are a senior software engineer and an expert in Python.
            Your role is to perform a meticulous code review.
            Critically evaluate the provided Python code based on the original task requirements.
            Look for bugs, style issues, missing edge cases, and areas for improvement.
            If the code is perfect and meets all requirements, respond with the single phrase 'CODE_IS_PERFECT'.
            Otherwise, provide a bulleted list of your critiques.
        """),
        HumanMessage(content=f"Original Task:\n{task_prompt}\n\nCode to Review:\n{current_code}")
    ]
    critique_response = llm.invoke(reflector_prompt)
    critique = critique_response.content

    # --- 3. STOPPING CONDITION ---
    if "CODE_IS_PERFECT" in critique:
        print("\n--- Critique ---\nNo further critiques found. The code is satisfactory.")
        break

    print("\n--- Critique ---\n" + critique)
    # Add the critique to the history for the next refinement loop.
    message_history.append(HumanMessage(content=f"Critique of the previous code:\n{critique}"))


========================= REFLECTION LOOP: ITERATION 1 =========================

>>> STAGE 1: GENERATING initial code...

--- Generated Code (v1) ---
Here's a Python function named `calculate_factorial` that meets the specified requirements:

```python
def calculate_factorial(n):
    """
    Calculate the factorial of a non-negative integer n.

    The factorial of a non-negative integer n is the product of all positive integers less than or equal to n.
    The factorial of 0 is defined to be 1.

    Parameters:
    n (int): A non-negative integer whose factorial is to be calculated.

    Returns:
    int: The factorial of the input integer n.

    Raises:
    ValueError: If the input is a negative integer.
    """
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    
    if n == 0:
        return 1
    
    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
    
    return factorial
```

### Explanation:

1. **Docstring**: The 